<a href="https://colab.research.google.com/github/CAVA1611/neural-networks/blob/main/TFM_Reddit_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción con Series Temporales de la carga en un centro de datos

**Autor:** Cesar Velasco Arias

In [1]:
# Importacion de librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import os
from time import sleep
import re 

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from statsmodels.tsa.seasonal import seasonal_decompose
import tensorflow as tf
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

np.random.seed(80)
random.seed(80)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)




/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path = '/content/drive/MyDrive/TFM/csv_2019'
year = '2019'

In [3]:

def files_verification():
  meses = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
  list_dir=[]
  files_to_read=[]
  files_not_found=[] 
  count = 0 
  try:
    list_dir = os.listdir(path)
    if os.path.isdir(path) and len(list_dir)>0:
      for mes in meses:
        file_to_read = 'LoadReddit-60minutes-'+ str(year)+'-'+ str(mes)+'-PLN.csv'
        if os.path.isfile(os.path.join(path, file_to_read)):
            count+=1
            files_to_read.append(file_to_read)
        else:
          files_not_found.append(file_to_read)
  except:  
    print('No se ha encontrado el directorio')
  
  print('Se han encontrado: {} correspondientes al año {}'.format(count,year))
  for f in files_to_read:
    print('-',f)
  print('\nHace falta {} archivos'.format(12-count))
  for nf in files_not_found:
    print('-',nf)
  print('\n')
  return files_to_read

In [4]:

def data_reading(path, year):
  global df
  meses = {'enero': '01', 'febrero':'02', 'marzo':'03', 'abril':'04', 'mayo':'05', 
           'junio':'06', 'julio':'07', 'agosto':'08','septiembre':'09',
           'octubre':'10', 'noviembre':'11', 'diciembre':'12'}
  month_conc = []
  files_to_read = files_verification()    
  for mes_k, mes_v in zip(meses.keys(), meses.values()):
    for file in files_to_read:
      pa = file.split('-')
      for p in pa:
        if p == mes_v:
          mes_k = pd.read_csv(os.path.join(path, file), 
                              header=None, names=['Date', 'number_of_post'], 
                              parse_dates=['Date']) 
          month_conc.append(mes_k)
          #print('Dimension para el mes {}: {}'.format(mes_k,mes_k.shape))
  df = pd.concat(month_conc, axis=0)
  print('*'*30)
  print('\nDimension de df -> ',df.shape)

  return (df, meses)

In [5]:
def graphic_1(start_month,end_month,year):
  global df_time, train_scaler, test_scaler
  
  df_time, meses = data_reading(path, year)
  df_time = df_time.set_index('Date')
  for mes_k, mes_v in zip(meses.keys(), meses.values()):
    if start_month == mes_k:
      start_month_index = mes_v
  for mes_k, mes_v in zip(meses.keys(), meses.values()):
    if end_month == mes_k:
      end_month_index = mes_v
  if start_month_index > end_month_index:
    print("Mal ingreso de mes, el mes de inicio no puede ser despues del segundo")
  else:
    print("Datos para graficar desde {} hasta {}".format(start_month, end_month))
    steps = int(len(df_time)*0.2)
    train = df_time[:-steps]
    test  = df_time[-steps:]
    max = train.max()
    min = train.min()
    train_scaler=  train['number_of_post'].apply(lambda x: (x-min)/(max-min))
    test_scaler= test['number_of_post'].apply(lambda x: (x-min)/(max-min)) 
    fig, (ax1, ax2)=plt.subplots(2, figsize=(20, 8))
    df_time.plot(ax=ax1, grid=True)
    train_scaler.plot(ax=ax2, grid=True)
    test_scaler.plot(ax=ax2, grid=True)
    ax1.legend(['Datos'])
    ax2.legend(['Train', 'Test'])
    ax1.set_title('Datos Originales')
    ax2.set_title('Datos Normalizados')
    plt.subplots_adjust(hspace=0.8)
  


In [9]:
def data_for_rn(df_train, df_test, n_obs):
  #n_obs -> es el numero de observacione que se toman para predecir un valor
  # Con los datos de train se debe dividir en un array "X_train" y  "y_train"; este es un formato necesario para la prediccion en series
  # temporales. En "X_train" se encontaran el numero de observaciones que leera el modelo para predecir y en "y_train" se encontrara
  # el valor a predecir. 
  # Ambos tendran la misma dimension en cuanto a numero de filas.
  # Para el caso de este trabajo se ha escogido como numero de observaciones 20, es decir que se leeran 20 datos para predecir 1

  Obs = n_obs
  X_train = np.atleast_3d(np.array([df_train[start:start + Obs] for start in range(0, df_train.shape[0] - Obs)]))
  y_train = df_train[Obs:]

  # Para probar el modelo entrenado con los datos de test se tiene que colocar en el mismo formato.
  X_test = np.atleast_3d(np.array([df_test[start:start + Obs] for start in range(0, df_test.shape[0] - Obs)]))
  y_test = df_test[Obs:]
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
  
  return (X_train, y_train, X_test, y_test)


  

## Lectura y Grafica de datos

In [ ]:
start_month = 'enero'
end_month = 'diciembre'
graphic_1(start_month,end_month,year)

In [12]:
# Obtener "X" y "y" train y test para las redes neuronales (solo es necesario realizar una vez)
# se trabaja con los datos escalados
(X_train, y_train, X_test, y_test) = data_for_rn(train_scaler, test_scaler, 20)
print("Dimension de X_train: ", X_train.shape)
print("\nDimension de y_train: ", y_train.shape)
print("\nDimension de X_test: ", X_test.shape)
print("\nDimension de y_test: ", y_test.shape)


Dimension de X_train:  (5813, 20, 1)

Dimension de y_train:  (5813, 1)

Dimension de X_test:  (1438, 20, 1)

Dimension de y_test:  (1438, 1)
